**Instruction:**

Explore and preprocess the billing history of the Tunisian electricity and gas company's customers from 2005 to 2019, focusing on understanding the data structure, handling missing values, performing descriptive analysis, and transforming categorical variables into numerical formats.

In [ ]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 1. Loading the dataset and displaying the first 10 rows
file_path = "/content/STEG_BILLING_HISTORY.csv"
df = pd.read_csv(file_path)
client_0_bills = df.head(10)
client_0_bills

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667.0,0,203,8.0,1,82,0,0,0,14302,14384,4,ELEC
1,train_Client_0,2013-03-29,11,1335667.0,0,203,6.0,1,1200,184,0,0,12294,13678,4,ELEC
2,train_Client_0,2015-03-23,11,1335667.0,0,203,8.0,1,123,0,0,0,14624,14747,4,ELEC
3,train_Client_0,2015-07-13,11,1335667.0,0,207,8.0,1,102,0,0,0,14747,14849,4,ELEC
4,train_Client_0,2016-11-17,11,1335667.0,0,207,9.0,1,572,0,0,0,15066,15638,12,ELEC
5,train_Client_0,2017-07-17,11,1335667.0,0,207,9.0,1,314,0,0,0,15638,15952,8,ELEC
6,train_Client_0,2018-12-07,11,1335667.0,0,207,9.0,1,541,0,0,0,15952,16493,12,ELEC
7,train_Client_0,2019-03-19,11,1335667.0,0,207,9.0,1,585,0,0,0,16493,17078,8,ELEC
8,train_Client_0,2011-07-22,11,1335667.0,0,203,9.0,1,1200,186,0,0,7770,9156,4,ELEC
9,train_Client_0,2011-11-22,11,1335667.0,0,203,6.0,1,1082,0,0,0,9156,10238,4,ELEC


In [ ]:
# 2. Checking the data type of 'client_0_bills'
data_type = type(client_0_bills)
print("Data Type of 'client_0_bills':\n", data_type)

Data Type of 'client_0_bills':
 <class 'pandas.core.frame.DataFrame'>


In [ ]:
# 3. Displaying the general information of the dataset
df_info = df.info()
print("Dataset Info:\n", df_info)

# How many rows and columns are in the dataset?
num_rows, num_columns = df.shape
print("\nNumber of Rows:", num_rows, "\nNumber of Columns:", num_columns)

# How many categorical features are present in the dataset?
categorical_features = df.select_dtypes(include=['object']).columns.tolist()
print("\n4 Categorical Features:\n", categorical_features)

# How much memory space does the dataset consume?
memory_usage = df.memory_usage(deep=True).sum()
print("\nMemory Usage (bytes):", memory_usage, "(422.7+ MB)")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4476749 entries, 0 to 4476748
Data columns (total 16 columns):
 #   Column                Dtype  
---  ------                -----  
 0   client_id             object 
 1   invoice_date          object 
 2   tarif_type            int64  
 3   counter_number        float64
 4   counter_statue        object 
 5   counter_code          int64  
 6   reading_remarque      float64
 7   counter_coefficient   int64  
 8   consommation_level_1  int64  
 9   consommation_level_2  int64  
 10  consommation_level_3  int64  
 11  consommation_level_4  int64  
 12  old_index             int64  
 13  new_index             int64  
 14  months_number         int64  
 15  counter_type          object 
dtypes: float64(2), int64(10), object(4)
memory usage: 546.5+ MB
Dataset Info:
 None

Number of Rows: 4476749 
Number of Columns: 16

4 Categorical Features:
 ['client_id', 'invoice_date', 'counter_statue', 'counter_type']

Memory Usage (bytes): 1499843451 

In [ ]:
# 4. Checking for missing values
missing_values = df.isnull().sum()
print("\nMissing Values:\n", missing_values)

df = df.dropna(subset=["counter_number"])     # Removing rows with missing "counter_number".

df["reading_remarque"].fillna(df["reading_remarque"].median(), inplace=True)  # Filling missing values in the "reading_remarque"  column with the median of the column.


Missing Values:
 client_id                  0
invoice_date               0
tarif_type                 0
counter_number            48
counter_statue             0
counter_code               0
reading_remarque        4531
counter_coefficient        0
consommation_level_1       0
consommation_level_2       0
consommation_level_3       0
consommation_level_4       0
old_index                  0
new_index                  0
months_number              0
counter_type               0
dtype: int64


1. I chose to remove rows with missing **"counter_number"** values because after using *df["counter_number"].unique()* to view the unique values, I noticed that the values of the column are unique identifiers for meters/counters.
Hence, imputation with a statistic (mean/median) would not suffice.
I also considered that removing 48 rows out of the total 4,476,749 rows would not affect my overall analysis.


2. I chose to fill missing values in the **"reading_remarque"**  column with the median of the column because after checking the value count using *df["reading_remarque"].value_counts(dropna=False)*, the result showed that it contains numerical values with outliers.
And, median is a better choice to fill the missing values with since it is less sensitive to outliers unlike the mean.

In [ ]:
# 6. Descriptive analysis on numerical features
numeric_summary = df.describe()
numeric_summary.T

,count,mean,std,min,25%,50%,75%,max
tarif_type,4476701.0,2.012810e+01,1.347258e+01,8.0,11.0,11.0,40.0,4.500000e+01
counter_number,4476701.0,1.230595e+11,1.657276e+12,0.0,121108.0,494561.0,1115161.0,2.798115e+13
counter_code,4476701.0,1.724879e+02,1.338872e+02,0.0,5.0,203.0,207.0,6.000000e+02
reading_remarque,4476701.0,7.322401e+00,1.571213e+00,5.0,6.0,8.0,9.0,4.130000e+02
counter_coefficient,4476701.0,1.003040e+00,3.083483e-01,0.0,1.0,1.0,1.0,5.000000e+01
consommation_level_1,4476701.0,4.109775e+02,7.573100e+02,0.0,79.0,274.0,600.0,9.999100e+05
consommation_level_2,4476701.0,1.093204e+02,1.220126e+03,0.0,0.0,0.0,0.0,9.990730e+05
consommation_level_3,4476701.0,2.030547e+01,1.574199e+02,0.0,0.0,0.0,0.0,6.449200e+04
consommation_level_4,4476701.0,5.292247e+01,8.754571e+02,0.0,0.0,0.0,0.0,5.479460e+05
old_index,4476701.0,1.776699e+04,4.036702e+04,0.0,1791.0,7690.0,21660.0,2.800280e+06


In [ ]:
# 7. Selecting records for 'train_Client_0' using two methods
# Method 1:
client_0_records_method1 = df[df['client_id'] == 'train_Client_0']
print("Client 'train_Client_0' Records (Method 1):\n", client_0_records_method1.head())

# Method 2:
client_0_records_method2 = df.loc[df['client_id'] == 'train_Client_0']
print("\n\n\nClient 'train_Client_0' Records (Method 2):\n", client_0_records_method2.head())

Client 'train_Client_0' Records (Method 1):
         client_id invoice_date  tarif_type  counter_number counter_statue  \
0  train_Client_0   2014-03-24          11       1335667.0              0   
1  train_Client_0   2013-03-29          11       1335667.0              0   
2  train_Client_0   2015-03-23          11       1335667.0              0   
3  train_Client_0   2015-07-13          11       1335667.0              0   
4  train_Client_0   2016-11-17          11       1335667.0              0   

   counter_code  reading_remarque  counter_coefficient  consommation_level_1  \
0           203               8.0                    1                    82   
1           203               6.0                    1                  1200   
2           203               8.0                    1                   123   
3           207               8.0                    1                   102   
4           207               9.0                    1                   572   

   consomma

In [ ]:
# 8. Encoding 'counter_type' using Label Encoding
encoder = LabelEncoder()
df['counter_type'] = encoder.fit_transform(df['counter_type'])
df.head()    # Dataset after Encoding and Column Deletion

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,train_Client_0,2014-03-24,11,1335667.0,0,203,8.0,1,82,0,0,0,14302,14384,4,0
1,train_Client_0,2013-03-29,11,1335667.0,0,203,6.0,1,1200,184,0,0,12294,13678,4,0
2,train_Client_0,2015-03-23,11,1335667.0,0,203,8.0,1,123,0,0,0,14624,14747,4,0
3,train_Client_0,2015-07-13,11,1335667.0,0,207,8.0,1,102,0,0,0,14747,14849,4,0
4,train_Client_0,2016-11-17,11,1335667.0,0,207,9.0,1,572,0,0,0,15066,15638,12,0


In [ ]:
# 9. Deleting 'counter_statue' feature
df.drop(columns=['counter_statue'], inplace=True)